In [1]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Bankloan.csv', encoding = 'cp1251', sep = ';')

In [3]:
# выводим первые 10 наблюдений датафрейма
data.head(10)

,age,job,employ,address,income,debtinc,creddebt,othdebt,default
0,28,working - other,7,2,44,"17,7","2,990592","4,797408",0
1,64,working - production,34,17,116,"14,7","5,047392","12,004608",0
2,40,working - IT,20,12,61,"4,8","1,042368","1,885632",0
3,30,working - IT,11,3,27,"34,5","1,75122","7,56378",0
4,25,working - IT,2,2,30,"22,4","0,75936","5,96064",1
5,35,working - IT,2,9,38,"10,9","1,462126","2,679874",1
6,26,own business,2,4,38,"11,9","0,954142","3,567858",1
7,25,working - IT,4,2,30,"14,4","1,04544","3,27456",0
8,65,civil service,29,14,189,5,"3,3642","6,0858",0
9,21,own business,0,0,23,"3,9","0,305877","0,591123",0


In [4]:
# смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
age         1500 non-null int64
job         1500 non-null object
employ      1500 non-null int64
address     1500 non-null int64
income      1500 non-null int64
debtinc     1500 non-null object
creddebt    1500 non-null object
othdebt     1500 non-null object
default     1500 non-null int64
dtypes: int64(5), object(4)
memory usage: 105.5+ KB


In [5]:
# заменяем запятые на точки и преобразуем в тип float
for i in ['debtinc', 'creddebt', 'othdebt']:
    data[i] = data[i].str.replace(',', '.').astype('float')
    
# выводим первые 5 наблюдений и смотрим типы переменных
print(data.head())
print('')
print(data.info())

   age                   job  employ  address  income  debtinc  creddebt  \
0   28       working - other       7        2      44     17.7  2.990592   
1   64  working - production      34       17     116     14.7  5.047392   
2   40          working - IT      20       12      61      4.8  1.042368   
3   30          working - IT      11        3      27     34.5  1.751220   
4   25          working - IT       2        2      30     22.4  0.759360   

     othdebt  default  
0   4.797408        0  
1  12.004608        0  
2   1.885632        0  
3   7.563780        0  
4   5.960640        1  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
age         1500 non-null int64
job         1500 non-null object
employ      1500 non-null int64
address     1500 non-null int64
income      1500 non-null int64
debtinc     1500 non-null float64
creddebt    1500 non-null float64
othdebt     1500 non-null float64
default     1500 non-null int

In [6]:
# создаем массив признаков и массив меток
y = data['default']
X = pd.get_dummies(data.loc[:, data.columns != 'default'])

## Обычная k-блочная перекрестная проверка

In [7]:
# импортируем функцию cross_val_score() и класс StratifiedKFold
from sklearn.model_selection import cross_val_score, StratifiedKFold
# создаем экземпляр класса StratifiedKFold
strat = StratifiedKFold(n_splits=10, shuffle=True, 
                        random_state=42)
# импортируем класс DecisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier
# создаем экземляр класса DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=152, max_depth=8)
# вычисляем значение правильности, усредненное по контрольным
# блокам обычной перекрестной проверки
scores_acc_tr = cross_val_score(tree, 
                                X, 
                                y, 
                                cv=strat)
print("Среднее значение правильности: {:.2f}".format(
    scores_acc_tr.mean()))

Среднее значение правильности: 0.71


In [8]:
# вычисляем значение правильности, усредненное по
# контрольным блокам повторной 
# перекрестной проверки
scores_auc_tr = cross_val_score(tree, 
                                X, y, 
                                scoring='roc_auc', 
                                cv=strat)
print("Среднее значение AUC: {:.2f}".format(
    scores_auc_tr.mean()))

Среднее значение AUC: 0.72


## Повторная k-блочная перекрестная проверка

In [9]:
# импортируем класс RepeatedKFold
from sklearn.model_selection import RepeatedKFold
# создаем экземпляр класса RepeatedKFold
rkf = RepeatedKFold(n_splits=10, n_repeats=10, 
                    random_state=42)
# вычисляем значение правильности, усредненное по контрольным
# блокам повторной перекрестной проверки
scores_acc_tr = cross_val_score(tree, 
                                X, 
                                y, 
                                cv=rkf)
print("Среднее значение правильности: {:.2f}".format(
    scores_acc_tr.mean()))

Среднее значение правильности: 0.71


## Перекрестная проверка с исключением по одному

In [10]:
# импортируем класс LeaveOneOut
from sklearn.model_selection import LeaveOneOut
# создаем экземпляр класса LeaveOneOut 
loo = LeaveOneOut()
# вычисляем значение правильности, усредненное по контрольным
# блокам перекрестной проверки c исключением по одному
scores_acc_tr = cross_val_score(tree, 
                                X, 
                                y, 
                                cv=loo)
print("Количество итераций cv: ", len(scores_acc_tr))
print("Среднее значение правильности: {:.2f}".format(
     scores_acc_tr.mean()))

Количество итераций cv:  1500
Среднее значение правильности: 0.71


## Комбинированная проверка в ручном режиме

In [11]:
# импортируем функцию train_test_split()
from sklearn.model_selection import train_test_split
# импортируем функцию roc_auc_score()
from sklearn.metrics import roc_auc_score
# создаем обучающий и тестовый массивы признаков и меток
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, 
                                                    stratify=y, random_state=42)
# создаем экземпляр класса StratifiedKFold, 5-блочную
# перекрестную проверку со случайным перемешиванием данных
strat = StratifiedKFold(n_splits=5, shufﬂe=True, random_state=42)
best_score = 0
for max_depth in [4, 6, 8, 10]:
    for max_features in [3, 6, 9, 12]:
        # для каждой из 16 комбинаций значений гиперпараметров создаем экземпляр
        # класса DecisionTreeClassifier - модель дерева CART
        tree = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features,  
                                      random_state=42)
        # для каждой из 16 комбинаций значений гиперпараметров будем строить 5 моделей дерева 
        # CART (5 раз на 4 обучающих блоках перекрестной проверки будем строить модель,
        # на 1 контрольном блоке - проверять, вычисляя AUC)
        scores = cross_val_score(tree, X_train, y_train, scoring='roc_auc', cv=strat)
        # для каждой из 16 комбинаций значений гиперпараметров вычисляем значение AUC, 
        # усредненное по 5 контрольным блокам перекрестной проверки 
        auc_score = np.mean(scores)
        # если получаем максимальное усредненное значение AUC, 
        # сохраняем его и комбинацию значений гиперпараметров
        if auc_score > best_score:
            best_score = auc_score
            best_parameters = {'max_features': max_features, 'max_depth': max_depth}
# строим модель с комбинацией значений гиперпараметров, давшей максимальное 
# усредненное значение AUC, на обучающей выборке
tree_best = DecisionTreeClassifier(**best_parameters, random_state=42)
tree_best.fit(X_train, y_train)
# проверяем качество модели с комбинацией значений гиперпараметров, 
# давшей максимальное усредненное значение AUC, на тестовой выборке
test_score = roc_auc_score(y_test, tree_best.predict_proba(X_test)[:, 1])
print("Наилучшие значения гиперпараметров: ", best_parameters)
print("Лучшее усредненное значение AUC cv: {:.2f}".format(best_score))
print("AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: {:.2f}".format(test_score))

Наилучшие значения гиперпараметров:  {'max_features': 9, 'max_depth': 4}
Лучшее усредненное значение AUC cv: 0.79
AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: 0.78


## Комбинированная проверка с помощью класса GridSearchCV

In [12]:
# создаем экземпляр класса DecisionTreeClassifier
tree_grid = DecisionTreeClassifier(random_state=42)
# импортируем класс GridSearchCV
from sklearn.model_selection import GridSearchCV
# задаем сетку параметров, будем перебирать разные значения штрафа
param_grid = {'max_depth': [4, 6, 8, 10], 
              'max_features': [3, 6, 9, 12]}
# создаем экземпляр класса GridSearchCV
grid_search = GridSearchCV(tree_grid, 
                           param_grid, 
                           scoring='roc_auc', 
                           return_train_score=True,
                           n_jobs=-1, cv=strat)
# запускаем решетчатый поиск
grid_search.fit(X_train, y_train)
# проверяем качество модели с комбинацией значений гиперпараметров, 
# давшей максимальное усредненное значение AUC, на тестовой выборке
test_score = roc_auc_score(y_test, grid_search.predict_proba(X_test)[:, 1])
# смотрим результаты решетчатого поиска
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))
print("Лучшее усредненное значение AUC cv: {:.2f}".format(grid_search.best_score_))
print("AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: {:.2f}".format(test_score))

Наилучшие значения гиперпараметров: {'max_depth': 4, 'max_features': 9}
Лучшее усредненное значение AUC cv: 0.79
AUC модели с наилучшими значениями гиперпараметров на тестовой выборке: 0.78


## Бутстреп

In [13]:
# импортируем необходимые функции
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
# конкатенируем массив признаков и массив меток
data = pd.concat([X, y], axis=1)
# преобразовываем датафрейм в массив NumPy
values = data.values
# задаем количество итераций
n_iterations = 100
n_size = int(len(data) * 0.50)
# создаем список, в котором будем 
# хранить значения правильности
acc_stats = list()
# создаем список, в котором будем 
# хранить значения AUC
auc_stats = list()
# запускаем бутстреп
for i in range(n_iterations):
    # подготавливаем обучающий и тестовый наборы
    train = resample(values, n_samples=n_size)
    test = np.array([x for x in values if x.tolist() not in train.tolist()])
    # обучаем модель дерева
    model = DecisionTreeClassifier()
    model.fit(train[:,:-1], train[:,-1])
    # оцениваем правильность модели
    predictions = model.predict(test[:,:-1])
    acc_score = accuracy_score(test[:,-1], predictions)
    acc_stats.append(acc_score)
    # оцениваем AUC модели
    probabilities = model.predict_proba(test[:,:-1])[:, 1]
    auc_score = roc_auc_score(test[:,-1], probabilities)
    auc_stats.append(auc_score)
# печатаем среднее значение правильности
print("Среднее значение правильности: {:.2f}".format(np.mean(acc_stats)))
# печатаем среднее значение AUC
print("Среднее значение AUC: {:.2f}".format(np.mean(auc_stats)))

Среднее значение правильности: 0.67
Среднее значение AUC: 0.65


## Доверительный интервал

### Асимптотический метод

#### Учимся вычислять асимптотический 90%-ный доверительный интервал среднего

In [14]:
# импортируем модуль random
import random
# задаем стартовое значение
# генератора случайных чисел
random.seed(444)
# генеририруем 50 случайных чисел
# в диапазоне от 45 до 100
income = np.random.uniform(45, 100, size=(50,))

In [15]:
# вычисляем среднее значение дохода
mean_income = np.mean(income)
mean_income

73.7432251233859

In [16]:
# записываем информацию о размере выборки
N = 50

In [17]:
# вычисляем предел погрешности 
err = 1.645 * (np.std(income) / np.sqrt(N))

In [18]:
# вычисляем нижнюю границу 90%-ного
# доверительного интервала 
mean_income - err

70.0359399671963

In [19]:
# вычисляем верхнюю границу 90%-ного
# доверительного интервала 
mean_income + err

77.4505102795755

### Бутстреп-метод

#### Учимся вычислять бутстрепированный 95%-ный доверительный интервал среднего

In [20]:
# импортируем модуль stats
from scipy import stats

# пишем функцию, извлекающую бутстреп-выборки
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

# пишем функцию, вычисляющую нижнюю и верхнюю границы доверительного 
# интервала для полученного распределения оценок
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

# пишем итоговую функцию, в которой объединяем первые две
def bootstrap_conf_int(data, stat_func, alpha=0.05, n_samples=1000):
    ''' 
    a = np.random.normal(size=1000)
    conf_int(a, np.median)
    '''
    scores = [stat_func(sample) for sample in get_bootstrap_samples(data, n_samples)]
    return stat_intervals(scores, alpha)

In [21]:
# вычисляем бутстрепированный 95%-ный 
# доверительный интервал среднего
bootstrap_conf_int(income, np.mean, alpha=0.05, n_samples=1000)

array([69.2450439 , 77.96877399])

### Доверительный интервал метрики качества (на примере AUC)

#### Учимся вычислять бутстрепированный 95%-ный доверительный интервал AUC и правильности

In [22]:
# вычисляем бутстрепированные доверительные интервалы
# правильности и AUC
alpha = 0.95
# задаем процентиль 2,5
p = ((1.0 - alpha) / 2.0) * 100
# вычисляем нижнюю границу доверительного интервала правильности
lower_acc = max(0.0, np.percentile(acc_stats, p))
# вычисляем нижнюю границу доверительного интервала AUC
lower_auc = max(0.0, np.percentile(auc_stats, p))
# задаем процентиль 97,5
p = (alpha + ((1.0 - alpha) / 2.0)) * 100
# вычисляем верхнюю границу доверительного интервала правильности
upper_acc = min(1.0, np.percentile(acc_stats, p))
# вычисляем верхнюю границу доверительного интервала AUC
upper_auc = min(1.0, np.percentile(auc_stats, p))
# печатаем доверительный интервал правильности
print("%.1f доверительный интервал правильности %.1f%% и %.1f%%" % (
    alpha * 100, lower_acc * 100, upper_acc * 100))
# печатаем доверительный интервал AUC
print("%.1f доверительный интервал AUC %.1f%% и %.1f%%" % (
    alpha * 100, lower_auc * 100, upper_auc * 100))

95.0 доверительный интервал правильности 63.8% и 70.0%
95.0 доверительный интервал AUC 61.2% и 67.7%
